### Brain Atlases

A brain atlas is a labeling of a volume. Each voxel contains a label, instead of a gray value. These labels are numeric. Each number corresponds to a region of interest (ROI).

Why ROIs?
1. Explore the data (see signal in areas of interest plotted for each condition or plotted against other variables of interest)
2. Limit number of statistical tests, controlling for Type I error
3. Limit statistical tests to a region that is functionally defined.

Some of these atlases are also explained here: https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/Atlases.

### Comparing Atlases

A full list of nilearn atlases can be found here: http://nilearn.github.io/modules/reference.html#module-nilearn.datasets.  
For questions about which brain atlas to use, a recent [Arslan et al., 2018 paper](https://www.ncbi.nlm.nih.gov/pubmed/28412442) may aid in your search.

### Retrieve the Atlas

Since we're using resting state fMRI data, it makes sense to use an atlas defined using resting state fMRI data.  
For this tutorial, we'll be using a set of parcellations from [Craddock et al., 2012](https://www.ncbi.nlm.nih.gov/pubmed/21769991). This atlas was generated from resting state fMRI data from 41 healthy controls participants. 

In [ ]:
import numpy as np
import nibabel as nib
from nilearn import datasets
from nilearn import image
from nilearn import plotting
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
parcel_dir = '../resources/rois/' 
fetch_atlas = datasets.fetch_atlas_craddock_2012(parcel_dir)

If we look at the dictionary keys for `fetch_atlas`, we can see that the Craddock atlas actually contains 5 different ROI atlases.

In [ ]:
fetch_atlas.keys()

Based on the paper, the `tcorr_2level` atlas (based on temporal correlation) performed better than the other methods.

Each atlas also contains 44 different volumes which correspond to the level of clustering. 

In [ ]:
cc_atlas = image.load_img(fetch_atlas['tcorr_2level'])
cc_atlas.shape

Let's have a look at that atlas.

In [ ]:
#Index 19, contains the 200 ROI parcellation we want to use
cc200 = cc_atlas.slicer[:, :, :, 19]
plotting.plot_roi(cc200, cmap='Paired')
print('Number of parcellations: ' + str(np.unique(cc200.get_data()).shape[0] -1))

In order to apply the atlas to our fMRI image, we first need to make sure that they have the same dimensions.

In [ ]:
func_file = '../data/ds000030/derivatives/fmriprep/sub-10171/func/sub-10171_task-rest_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'
func_img = nib.load(func_file)
func_img.shape[0:2]

In [ ]:
cc200.get_data().shape[0:2]

In [ ]:
mean_func_img = image.mean_img(func_file)

The dimensions don't match so we'll resample the atlas to match.

In [ ]:
resampled_cc200 = image.resample_to_img(cc200, mean_func_img, interpolation = 'nearest')
plotting.plot_roi(resampled_cc200, mean_func_img)